In [14]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, AveragePooling2D, Conv2D, Flatten, GlobalAveragePooling2D
from keras.layers import Lambda
from keras import backend as K # tensorflow

import numpy as np

import coremltools
from coremltools.proto import NeuralNetwork_pb2

import os

In [15]:
kears_file = "./KerasMNIST_customlayer.h5"
coreml_file = './KerasMNIST_customlayer.mlmodel'

In [26]:
# sigmoid
def MySigmoid(x):
    return K.sigmoid(x)
# relu
def MyRelu(x):
    return K.relu(x)

In [27]:
def build_and_learn_keras_model():

    batch_size = 128
    num_classes = 10
    epochs = 20

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    print(x_train.shape)
    print(x_train.shape[1:])

    img_rows = 28
    img_cols = 28

    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # 入力値の正規化
    x_train /= 255
    x_test /= 255

    # 教師データをクラス分類のデータに変換
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # ネットワーク設計
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[6, 6], padding='same', input_shape=input_shape))

    # custom layer
    model.add(Lambda(MyRelu))
    
    model.add(Conv2D(32, kernel_size=[6, 6], padding='same', input_shape=input_shape))
    model.add(Lambda(MyRelu))
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', input_shape=input_shape))
    model.add(Lambda(MySigmoid))
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128))
    model.add(Activation('relu'))

    model.add(Dense(num_classes, activation='softmax'))

    # ネットワークの構成を出力する
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [28]:
keras_model = build_and_learn_keras_model()
keras_model.save(kears_file)

(60000, 28, 28)
(28, 28)
(60000, 'train samples')
(10000, 'test samples')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 28, 28, 32)        1184      
_________________________________________________________________
lambda_13 (Lambda)           (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 32)        36896     
_________________________________________________________________
lambda_14 (Lambda)           (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
lambda_15 (Lambda)           (None, 28, 28, 32)        0         
_________________________________________________________________
co

In [31]:
def convert_custom_layer(layer):
    
    params = NeuralNetwork_pb2.CustomLayerParams()
    
    if layer.function.__name__ == MySigmoid.__name__:
        # カスタムレイヤーに割り当てるクラス
        params.className = "MyCustomSigmoid"
        # Xcodeで表示される解説文
        params.description = "sigmoid"
        
        # その他のパラメタ
        params.parameters["test"].stringValue = "hoge"
        params.parameters["param"].intValue = 10
        
        # 重みを設定・・・ReLUには無関係
        my_weights = params.weights.add()
        my_weights.floatValue.extend(np.zeros(10).astype(float))
        my_weights.floatValue[0] = 3
        my_weights.floatValue[1] = 1
        my_weights.floatValue[2] = 4
        my_weights.floatValue[3] = 1
        my_weights.floatValue[4] = 5
        my_weights.floatValue[5] = 9
        my_weights.floatValue[6] = 2
        my_weights.floatValue[7] = 6
        my_weights.floatValue[8] = 5
        my_weights.floatValue[9] = 3
        
        return params
    
    if layer.function.__name__ == MyRelu.__name__:
        # カスタムレイヤーに割り当てるクラス
        params.className = "MyCustomReLU"
        # Xcodeで表示される解説文
        params.description = "relu"
        
        return params
    else:
        return None

In [32]:
coreml_model = coremltools.converters.keras.convert(
    keras_model,
    input_names='image',
    output_names='digit',
    add_custom_layers=True,
    custom_conversion_functions={ "Lambda": convert_custom_layer }
)

coreml_model.author = u'Yuichi Yoshida'
coreml_model.license = 'MIT'
coreml_model.short_description = u'Custom layerのサンプル．'

coreml_model.input_description['image'] = u'入力画像'
coreml_model.output_description['digit'] = u'推定した数字の確率'

coreml_model.save(coreml_file)

# Look at the layers in the converted Core ML model.
print("\nLayers in the converted model:")
for i, layer in enumerate(coreml_model._spec.neuralNetwork.layers):
    if layer.HasField("custom"):
        print("Layer %d = %s --> custom layer = %s" % (i, layer.name, layer.custom.className))
    else:
        print("Layer %d = %s" % (i, layer.name))

0 : conv2d_25_input, <keras.engine.topology.InputLayer object at 0x14a5eee50>
1 : conv2d_25, <keras.layers.convolutional.Conv2D object at 0x14a5eeed0>
2 : lambda_13, <keras.layers.core.Lambda object at 0x14a5eef50>
3 : conv2d_26, <keras.layers.convolutional.Conv2D object at 0x14a5ee5d0>
4 : lambda_14, <keras.layers.core.Lambda object at 0x14a609a10>
5 : conv2d_27, <keras.layers.convolutional.Conv2D object at 0x14a638390>
6 : lambda_15, <keras.layers.core.Lambda object at 0x14a638790>
7 : conv2d_28, <keras.layers.convolutional.Conv2D object at 0x149cce890>
8 : global_average_pooling2d_7, <keras.layers.pooling.GlobalAveragePooling2D object at 0x149ccef10>
9 : dense_13, <keras.layers.core.Dense object at 0x14300cb10>
10 : activation_15, <keras.layers.core.Activation object at 0x14981a4d0>
11 : dense_14, <keras.layers.core.Dense object at 0x14a3aff10>
12 : dense_14__activation__, <keras.layers.core.Activation object at 0x14a50a1d0>

Layers in the converted model:
Layer 0 = conv2d_25
Layer 